In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### requiere la carpeta de cora de https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
### cora.cites es un grafo que une a dos paper_id cuando uno cita al otro


## Dataset de CORA

Este dataset contiene dos archivos. El primero es una base de datos de papers que citan a otros papers

In [4]:
citas = pd.read_csv('cora/cora.cites',sep="\t",
    header=None,
    names=["target", "source"])

citas

,target,source
0,35,1033
1,35,103482
2,35,103515
3,35,1050679
4,35,1103960
...,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289
5427,853155,853118


El segundo archivo contiene informacién de muchas palabras (términos) asociadas a cada paper, y además una categoría. 

In [5]:
###informacion de los papers de acuerdo a loas palabras que mencionan, junto a su paper id y el tema general

column_names = ["paper_id"] + [f"word_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv(
    'cora/cora.content', sep="\t", names=column_names,
)
papers

,paper_id,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,...,word_1424,word_1425,word_1426,word_1427,word_1428,word_1429,word_1430,word_1431,word_1432,subject
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,1128975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2704,1128977,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2705,1128978,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2706,117328,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based


In [6]:
print(papers.subject.value_counts())


subject
Neural_Networks           818
Probabilistic_Methods     426
Genetic_Algorithms        418
Theory                    351
Case_Based                298
Reinforcement_Learning    217
Rule_Learning             180
Name: count, dtype: int64


Vamos a pasar estos subjects a números, y armar los paper_id para que sean consecutivos

In [7]:
class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citas["source"] = citas["source"].apply(lambda name: paper_idx[name])
citas["target"] = citas["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

In [8]:
citas

,target,source
0,0,21
1,0,905
2,0,906
3,0,1909
4,0,1940
...,...,...
5424,1873,328
5425,1873,1876
5426,1874,2586
5427,1876,1874


In [9]:
print(papers.subject.value_counts())

subject
2    818
3    426
1    418
6    351
0    298
4    217
5    180
Name: count, dtype: int64


### Sets de training y test
Lo último antes de empezar: dividir en dos este dataset (y tratar que nos queden bien balanceados los subjects)

In [10]:
#Primero sacamos un 50% de nodos de cada subject
train_data, test_data = [], []

for nombres, datos_agrupados in papers.groupby("subject"):
    random_selection = np.random.rand(len(datos_agrupados.index)) <= 0.5
    train_data.append(datos_agrupados[random_selection])
    test_data.append(datos_agrupados[~random_selection])
train_data = pd.concat(train_data)
test_data = pd.concat(test_data)

In [11]:
train_data

,paper_id,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,...,word_1424,word_1425,word_1426,word_1427,word_1428,word_1429,word_1430,word_1431,word_1432,subject
13,1233,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,2658,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,1237,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,2659,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,2660,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2651,1319,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2685,900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2689,1019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2695,1081,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


Nuestra tarea consiste en predecir la línea "subject" (el tema del paper), basándonos por ahora solo en el vector de ocurrencias de palabras. Con esto, dividimos el set de training y test en una matriz x y un vector y con los subjects. 

In [13]:
# Y luego algunas cosas que se necesitan    
feature_names = list(set(papers.columns) - {"paper_id", "subject"})
num_features = len(feature_names)
num_classes = len(class_idx)

# A keras le gustan los arreglos numpy
x_train = train_data[feature_names].to_numpy()
x_test = test_data[feature_names].to_numpy()

y_train = train_data["subject"]
y_test = test_data["subject"]

In [14]:
x_train, y_train

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 13      0
 44      0
 47      0
 49      0
 50      0
        ..
 2651    6
 2685    6
 2689    6
 2695    6
 2696    6
 Name: subject, Length: 1332, dtype: int64)

## La clase modelo en Keras

Para definir capas en una GNN tenemos que extender la clase *model* en Keras. Para orientarnos con esta funcionalidad vamos a crear primero una red neuronal básica que nos permita comparar resultados. La idea es que esta red básica sirva para predecir los temas de los papers usando solamente los features de cada paper (nodo en el grafo), y no sus conexiones. 


### Un perceptron multicapa (multilayered perceptron, MLP)
Esta primera función nos va a retornar un objeto Keras. Recibe un arreglo *capas_internas* de una dimension ([32,32]) y entrega un MLP completamente conectado, con capas de esas dimensiones, y con operaciones usuales (normalizaciòn, dropout, activaciòn no lineal) en cada capa. 

In [15]:
def create_MLP(capas_internas, dropout_rate, name=None):
    capas = []
    num_capa = 0
    
    for capa_interna in capas_internas:
        capas.append(layers.BatchNormalization())
        capas.append(layers.Dropout(dropout_rate))
        capas.append(layers.Dense(capa_interna, activation=tf.nn.gelu))

    return keras.Sequential(capas, name=name)


### Un primer clasificador. 

Recordemos que la tarea es predecir el subject de un paper, basado en el vector de ocurrencias de palabras. 
Nuestro clasificador tiene una primera capa para pasar de las dimensiones del input a la dimension de *capas_internas*, tres MLP conectados y una última capa para hacer la predicción. Los MLP están conectados secuencialmente, pero a su vez establecemos unas "skip connections" en el grafo de cómputo final. Tensorflow se encarga por detrás de hacer que todo propague bien para el entrenamiento. 

In [16]:
class BaseClassifier(tf.keras.Model):
    def __init__(
        self,
        num_classes,
        capas_internas = [32,32],
        dropout_rate=0.5,
        normalize=True,
        *args,
        **kwargs,
    ):
        super(BaseClassifier, self).__init__(*args, **kwargs)

        
        ### Definimos las capas de nuestro modelo como atributos. 
        ## Una capa que preprocese node_features y los lleve a una dimensionalidad mas pequeña
        
        self.preprocesador = create_MLP(capas_internas, dropout_rate, name="preprocesador")

        ## Tres capas adicionales 
        
        self.capa1 = create_MLP(capas_internas, dropout_rate, name="capa1")
        self.capa2 = create_MLP(capas_internas, dropout_rate, name="capa2")
        self.capa3 = create_MLP(capas_internas, dropout_rate, name="capa3")

        ## La ultima capa que llegue a num_clases 

        #self.clasificador = layers.Dense(units=num_classes, name="clasificador", activation=tf.keras.activations.softmax)
        self.clasificador = layers.Dense(units=num_classes, name="clasificador")
        
    ### La función call es la que se llama cuando se entrena o cuando se quiere predecir con una red ya entrenada. 
    ### Aquí es donde tenemos que explicarle a tensorflow como hacer las operaciones. 
    
    def call(self,input_features):
            
        i = self.preprocesador(input_features)
        c1 = self.capa1(i)
            
            #skip connection
        s1 = c1 + i 
            
            #otra capa, skip, capa, skip
        c2 = self.capa2(s1)
        s2 = c2 + s1 
        c3 = self.capa3(s2)
        s3 = c3 + s2
        

        return self.clasificador(s3)
        
        
        

### Entrenando el modelo

In [17]:
#Creamos un objeto BaseClassifier, tenemos 7 categorías de papers. 

BASE = BaseClassifier(
    num_classes=7,
    name="modelo_base",
)

In [18]:
#Compilamos el clasificador. Establecemos la learning rate, el loss y la métrica. Esto es estandar para un 
#clasificador discreto
BASE.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

In [19]:
# Early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_acc", patience=50, restore_best_weights=True
)

#Y finalmente ajustamos
history = BASE.fit(
    x=x_train,
    y=y_train,
    epochs=300,
    batch_size=256,
    validation_split=0.15,
    callbacks=[early_stopping],
)

Epoch 1/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - acc: 0.2780 - loss: 1.8247 - val_acc: 0.0000e+00 - val_loss: 3.2589
Epoch 2/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7201 - loss: 1.2503 - val_acc: 0.0000e+00 - val_loss: 6.2589
Epoch 3/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8268 - loss: 0.5444 - val_acc: 0.0850 - val_loss: 6.5239
Epoch 4/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9064 - loss: 0.2843 - val_acc: 0.0150 - val_loss: 8.9175
Epoch 5/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.9451 - loss: 0.1560 - val_acc: 0.0800 - val_loss: 9.6673
Epoch 6/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9807 - loss: 0.0672 - val_acc: 0.1250 - val_loss: 8.9208
Epoch 7/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.9870 - loss: 0.0359 - val_acc: 0.0750 - val_loss: 13.1536
Epoch 8/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.9898 - loss: 0.0231 - val_acc: 0.0700 - val_loss: 13.3551
Epoch 9/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9953 - loss

In [20]:
BASE.summary()

Model: "modelo_base"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ preprocesador (Sequential)      │ (None, 32)             │        52,804 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa1 (Sequential)              │ (None, 32)             │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa2 (Sequential)              │ (None, 32)             │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa3 (Sequential)              │ (None, 32)             │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ clasificador (Dense)            │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 173,791 (678.88 KB)

 Trainable params: 56,825 (221.97 KB)

 Non-trainable params: 3,314 (12.95 KB)

 Optimizer params: 113,652 (443.96 KB)

In [21]:
BASE.evaluate(x=x_test, y=y_test)

43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - acc: 0.7076 - loss: 2.3933


[4.030873775482178, 0.6497092843055725]

Malazo el modelo este 